In [ ]:
# Drone Delivery Debrecen with No-Fly Zones

import pandas as pd
from shapely.geometry import Polygon, Point
import folium
import random
import googlemaps
import json

# Load Google Maps API key from a protected file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)
GOOGLE_MAPS_API_KEY = config['GOOGLE_MAPS_API_KEY']

DEBRECEN_POLYGON_MAP_COORDINATES_PATH = 'coordinates/debrecen_coordinates_v1.csv'
NO_FLY_ZONE_COORDINATES_PATHS = [
    'coordinates/airport_coordinates_v1.csv',
    'coordinates/clinics_coordinates_v1.csv',
    'coordinates/kenezi_coordinates_v1.csv',
    'coordinates/mainsquare_coordinates_v1.csv',
    'coordinates/military_coordinates_v1.csv',
    'coordinates/railwaystation_coordinates_v1.csv',
    'coordinates/stadium_coordinates_v1.csv'
]
NUMBER_OF_SIMULATED_ORDERS = 10

FORUM_COORDINATE_X = 47.5326511
FORUM_COORDINATE_Y = 21.6287677

PLOT_ROUTES_ON_MAP = False

# Load Debrecen polygon coordinates
data = pd.read_csv(DEBRECEN_POLYGON_MAP_COORDINATES_PATH)
coordinates = list(zip(data['Latitude'], data['Longitude']))
debrecen_polygon = Polygon(coordinates)

# Load no-fly zones
no_fly_zones = []
for path in NO_FLY_ZONE_COORDINATES_PATHS:
    no_fly_data = pd.read_csv(path)
    no_fly_coordinates = list(zip(no_fly_data['Latitude'], no_fly_data['Longitude']))
    no_fly_zones.append(Polygon(no_fly_coordinates))

# Generate the base map
center_lat = data['Latitude'].mean()
center_lon = data['Longitude'].mean()
map_debrecen = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Add Debrecen polygon to the map
folium.Polygon(
    locations=coordinates,
    color='blue',
    fill=True,
    fill_color='lightblue',
    fill_opacity=0.5,
    weight=2
).add_to(map_debrecen)

# Add no-fly zones to the map
for no_fly_zone in no_fly_zones:
    folium.Polygon(
        locations=list(no_fly_zone.exterior.coords),
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.5,
        weight=2
    ).add_to(map_debrecen)

# Generate random points inside Debrecen polygon excluding no-fly zones
min_x, min_y, max_x, max_y = debrecen_polygon.bounds
random_points_inside_polygon = []

while len(random_points_inside_polygon) < NUMBER_OF_SIMULATED_ORDERS:
    random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
    if debrecen_polygon.contains(random_point) and not any(zone.contains(random_point) for zone in no_fly_zones):
        random_points_inside_polygon.append((random_point.x, random_point.y))

# Add random points to the map
for point in random_points_inside_polygon:
    folium.CircleMarker(
        location=point,
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
    ).add_to(map_debrecen)

# Mark Fórum Debrecen on the map
forum_point = Point(FORUM_COORDINATE_X, FORUM_COORDINATE_Y)
forum_point_data = (forum_point.x, forum_point.y)
folium.CircleMarker(
    location=forum_point_data,
    radius=8,
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.9,
).add_to(map_debrecen)

# Initialize Google Maps API client
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

# Calculate routes and durations for each destination
results = []
for idx, destination in enumerate(random_points_inside_polygon):
    modes = ['driving', 'walking', 'bicycling']
    route_info = {'Destination': idx + 1, 'Coordinates': destination}

    for mode in modes:
        directions = gmaps.directions(
            origin=forum_point_data,
            destination=destination,
            mode=mode
        )

        if directions:
            leg = directions[0]["legs"][0]
            route_info[f'{mode}_duration'] = leg["duration"]["text"]
            route_info[f'{mode}_distance'] = leg["distance"]["text"]
        else:
            route_info[f'{mode}_duration'] = 'N/A'
            route_info[f'{mode}_distance'] = 'N/A'

    results.append(route_info)

# Plot driving routes on the map if enabled
if PLOT_ROUTES_ON_MAP:
    map_plot = folium.Map(location=[FORUM_COORDINATE_X, FORUM_COORDINATE_Y], zoom_start=13)

    # Add Fórum Debrecen marker
    folium.Marker(location=forum_point_data, popup="Start Point", icon=folium.Icon(color="blue")).add_to(map_plot)

    # Add destinations and driving routes
    for result in results:
        folium.Marker(
            location=result['Coordinates'],
            popup=f"Destination {result['Destination']}",
            icon=folium.Icon(color="red")
        ).add_to(map_plot)

        # Plot driving route
        directions = gmaps.directions(
            origin=forum_point_data,
            destination=result['Coordinates'],
            mode='driving'
        )

        if directions:
            steps = directions[0]['legs'][0]['steps']
            route_coordinates = [(step['start_location']['lat'], step['start_location']['lng']) for step in steps]
            route_coordinates.append(result['Coordinates'])
            folium.PolyLine(route_coordinates, color='green', weight=2.5, opacity=0.8).add_to(map_plot)

    #map_plot.save("driving_routes_map.html")

# Save results to a DataFrame
results_df = pd.DataFrame(results)

#map_debrecen

FileNotFoundError: [Errno 2] No such file or directory: 'config.json'

In [11]:
results_df

,Destination,Coordinates,driving_duration,driving_distance,walking_duration,walking_distance,bicycling_duration,bicycling_distance
0,1,"(47.51072491499786, 21.61717614000003)",10 mins,4.1 km,41 mins,3.1 km,10 mins,3.1 km
1,2,"(47.512009996943235, 21.69135581604597)",14 mins,6.6 km,1 hour 23 mins,6.0 km,21 mins,6.5 km
2,3,"(47.49455547392202, 21.634398081681148)",12 mins,5.3 km,1 hour 3 mins,4.6 km,18 mins,5.1 km
3,4,"(47.52176706082439, 21.65974540153322)",11 mins,4.1 km,44 mins,3.2 km,12 mins,3.2 km
4,5,"(47.546960267393374, 21.66950337616714)",14 mins,5.2 km,57 mins,4.1 km,16 mins,4.5 km
5,6,"(47.52441605098242, 21.703900388856393)",15 mins,7.0 km,1 hour 25 mins,6.2 km,23 mins,7.0 km
6,7,"(47.51068251232579, 21.67068677126784)",12 mins,5.4 km,1 hour 3 mins,4.6 km,17 mins,4.8 km
7,8,"(47.5104920646398, 21.62502186584659)",9 mins,3.5 km,39 mins,2.8 km,14 mins,3.3 km
8,9,"(47.55514373371019, 21.665833119312623)",13 mins,4.8 km,55 mins,3.9 km,15 mins,4.2 km
9,10,"(47.529434912915995, 21.588328596742752)",11 mins,4.9 km,48 mins,3.5 km,11 mins,3.5 km


In [12]:
map_debrecen